# Direct S3 Data Access

## Summary  

In the previous exercises we searched for and discovered cloud data assets that met certain search criteria (i.e., intersects with our region of interest and for a specified date range). The end goal was to find and save web links to the data assets we want to use in our workflow. The links we found allow us to download data via HTTPS (Hypertext Transfer Protocol Secure). However, NASA allows for direct in-region S3 bucket access for the same assets. In addition to saving the HTTPS links, we also created and saved the S3 links for those same cloud assets and we will use them here. In this exercise we will demonstrate how to perform direction in-region S3 bucket access for Harmonized Landsat Sentinel-2 (HLS) cloud data assets.  

### Direct S3 Access  

NASA Eartdata Cloud provides two pathways for accessing data from the cloud. The first is via HTTPS. The other is through direct S3 bucket access. Below are some benefits and considerations when choosing to use direct S3 bucket access for NASA cloud assets.  

#### Benefits  

- Retrieve data is very quickly  
- No need to download data! Work with data in a more efficient manner  
- Increased capacity to do parallel processing  
- You are working completely with the AWS cloud ecosystem and thus have access to the might of all AWS offerings (e.g., infrastructure, S3 API, services, etc.)  

#### Considerations  

- If you're workflow is in the cloud, choose S3 over HTTPS  
- Access only works within AWS us-west-2 region  
- Need an AWS S3 “token” to access S3 Bucket  
- Token expires after 1 hour  
- Token only works at the DAAC that generates it, e.g.,  
    - PO.DAAC token generator: <https://archive.podaac.earthdata.nasa.gov/s3credentials>  
    - LP DAAC token generator: <https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials>  
- S3 on its own does not solve ‘cloud’ problems, but it is one key technology in solving big data problems  
- Still have to load things in to memory, parallelize the computation, if working with really large data volumes. There are a lot of tool that allow you to do that, not discussed in this tutorial  

## What you will learn from this tutorial   

- how to retrieve temporary S3 credentials for in-region direct S3 bucket access  
- how to configure our notebook environment for in-region direct S3 bucket access  
- how to access a single HLS file via in-region direct S3 bucket access  
- how to create an HLS time series data array from cloud assets via in-region direct S3 bucket access  
- how to plot results  

---

## Import Required Packages

In [ ]:
#

## Configure Local Environment and Get Temporary Credentials  

To perform direct S3 data access one needs to acquire temporary S3 credentials. The credentials give users direct access to S3 buckets in NASA Earthdata Cloud. **AWS credentials should not be shared**, so take precautions when using them in notebooks our scripts. **Note,** these temporary credentials are valid for only **1 hour**. For more information regarding the temporary credentials visit <https://data.lpdaac.earthdatacloud.nasa.gov/s3credentialsREADME>. A `netrc` file is required to aquire these credentials. Use the [NASA Earthdata Authentication](NASA_Earthdata_Authentication.ipynb) to create a `netrc` file in your home directory.

In [ ]:
#

In [ ]:
#

In [ ]:
#

### Insert the credentials into our `boto3` session and configure our `rasterio` environment for data access

Create a boto3 Session object using your temporary credentials. This Session is used to pass credentials and configuration to AWS so we can interact wit S3 objects from applicable buckets.

In [ ]:
#

For this exercise, we are going to open up a context manager for the notebook using the `rasterio.env` module to store the required GDAL and AWS configurations we need to access the data in Earthdata Cloud. While the context manager is open (`rio_env.__enter__()`) we will be able to run the open or get  data commands that would typically be executed within a `with` statement, thus allowing us to more freely interact with the data. We’ll close the context (`rio_env.__exit__()`) at the end of the notebook.

GDAL environment variables must be configured to access Earthdata Cloud data assets. Geospatial data access Python packages like `rasterio` and `rioxarray` depend on GDAL, leveraging GDAL's "Virtual File Systems" to read remote files. GDAL has a lot of environment variables that control it's behavior. Changing these settings can mean the difference being able to access a file or not. They can also have an impact on the performance.

In [ ]:
#

## Read in S3 Links

In the [CMR-STAC API](Data_Discovery__CMR-STAC_API.ipynb) tutorial we saved off multiple text file containing links, both HTTPS and S3 links, to Harmonized Landsat Sentinel-2 (HLS) cloud data assets. We will now read in one of those file and show how to access those data assets.

### List the available files in the data directory

In [ ]:
#

We will safe our list of links and a single link as Python objects for use later.

In [ ]:
#

In [ ]:
#

## Read in a single HLS file

We'll access the HLS S3 object using the [`rioxarray`](https://corteva.github.io/rioxarray/stable/) Python package. The package is an extension of `xarray` and `rasterio`, allowing users to read in and interact with geospatial data using xarray data structures. We will also be leveraging the tight integration between xarray and dask to lazily read in data via the `chunks` parameter. This allows us to connect to the HLS S3 object, reading only metadata, an not load the data into memory until we request it via the `loads()` function. 

In [ ]:
#

When GeoTIFFS/Cloud Optimized GeoTIFFS are read in, a `band` coordinate variable is automatically created (see the print out above). In this exercise we will not use that coordinate variable, so we will remove it using the `squeeze()` function to avoid confusion. 

In [ ]:
#

### Plot the HLS S3 object

In [ ]:
#

We can print out the data value as a numpy array by typing `.values`

In [ ]:
#

Up to this point, we have not saved anything but metadata into memory. To save or load the data into memory we can call the `.load()` function. 

In [ ]:
#

In [ ]:
#

## Read in HLS as a time series

Now we'll read in multiple HLS S3 objects as a time series xarray. Let's print the links list again to see what we're working with.

In [ ]:
#

Currently, the utilities and packages used in Python to read in GeoTIFF/COG file do not recognize associated dates stored in the internal metadata. To account for the dates for each file we must create a time variable and add it as a dimension in our final time series xarray. We'll create a function that extracts the date from the file link and create an xarray variable with a time array of datetime objects. 

In [ ]:
#

In [ ]:
#

We'll now specify a chunk size to use that matches the internal tiling of HLS files. This will help improve performance.

In [ ]:
#

Now, we will create our time series.

In [ ]:
#

Since we used the `chunks` parameter while reading the data, the `hls_ts_da` object is not read into memory yet. To do that we'll use the `load()` function.

Now, we'll see what we have. Use `hvplot` to plot our time series

In [ ]:
#

In [ ]:
#

In [ ]:
#

## Concluding Remarks

The above exercise demonstrated how to perform in-region direct S3 bucket access for HLS cloud data assets. HLS cloud data assets are stored as [Cloud Optimized GeoTIFFs](https://www.cogeo.org/), a format that has been the benifactor of data discovery and access advancements within the Python ecosystem. Knowing what the data storage format is (e.g., COG, netcdf4, or zarr store) and/or what data access protocol you're using is critical in determining what Python data access method you will use. For COG data, `rioxarray` package is often prefered due to is ability to bring the geospatial data format into an `xarray` object. For netcdf4 files, the standard `xarray` package incombination with `s3fs` allow users to perform in-region direct access reads into an xarray object. Finally, if you are using OPeNDAP to connect to data, specialized packages like `pydap` have been integrated into `xarray` for streamline access directly to an `xarray` object. 